In [1]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0

In [2]:
import time
import random
import pandas as pd
from glob import glob
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig

In [3]:
MAX_SEQ_LENGTH=128
BATCH_SIZE=16
EPOCHS=1
STEPS_PER_EPOCH=250
VALIDATION_STEPS=50
TEST_STEPS=50
CLASSES = [1, 2, 3, 4, 5]
NUM_GPUS=0

In [4]:
def select_data_and_label_from_record(record):
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
    }
    y = record['label_ids']

    return (x, y)

In [5]:
def file_based_input_dataset_builder(channel,
                                     input_filenames,
                                     pipe_mode,
                                     is_training,
                                     drop_remainder):

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.

    if pipe_mode:
        print('***** Using pipe_mode with channel {}'.format(channel))
        from sagemaker_tensorflow import PipeModeDataset
        dataset = PipeModeDataset(channel=channel,
                                  record_format='TFRecord')
    else:
        print('***** Using input_filenames {}'.format(input_filenames))
        dataset = tf.data.TFRecordDataset(input_filenames)

    dataset = dataset.repeat(EPOCHS * STEPS_PER_EPOCH)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    name_to_features = {
      "input_ids": tf.io.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "input_mask": tf.io.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "segment_ids": tf.io.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "label_ids": tf.io.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        return tf.io.parse_single_example(record, name_to_features)
        
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
          lambda record: _decode_record(record, name_to_features),
          batch_size=BATCH_SIZE,
          drop_remainder=drop_remainder,
          num_parallel_calls=tf.data.experimental.AUTOTUNE))

    dataset.cache()

    if is_training:
        dataset = dataset.shuffle(seed=42,
                                  buffer_size=10,
                                  reshuffle_each_iteration=True)

    return dataset

In [6]:
train_data = './data-tfrecord/bert-train'

In [7]:
train_data_filenames = glob('{}/*.tfrecord'.format(train_data))
print('train_data_filenames {}'.format(train_data_filenames))
train_dataset = file_based_input_dataset_builder(
    channel='train',
    input_filenames=train_data_filenames,
    pipe_mode=False,
    is_training=True,
    drop_remainder=False).map(select_data_and_label_from_record)

train_data_filenames ['./data-tfrecord/bert-train/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-train/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']
***** Using input_filenames ['./data-tfrecord/bert-train/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-train/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.


In [8]:
validation_data = './data-tfrecord/bert-validation'

In [9]:
validation_data_filenames = glob('{}/*.tfrecord'.format(validation_data))

print('validation_data_filenames {}'.format(validation_data_filenames))
validation_dataset = file_based_input_dataset_builder(
    channel='validation',
    input_filenames=validation_data_filenames,
    pipe_mode=False,
    is_training=False,
    drop_remainder=False).map(select_data_and_label_from_record)

validation_data_filenames ['./data-tfrecord/bert-validation/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-validation/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']
***** Using input_filenames ['./data-tfrecord/bert-validation/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-validation/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']


In [10]:
test_data = './data-tfrecord/bert-test'

In [11]:
test_data_filenames = glob('{}/*.tfrecord'.format(test_data))

print(test_data_filenames)

test_dataset = file_based_input_dataset_builder(
    channel='test',
    input_filenames=test_data_filenames,
    pipe_mode=False,
    is_training=False,
    drop_remainder=False).map(select_data_and_label_from_record)

['./data-tfrecord/bert-test/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-test/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']
***** Using input_filenames ['./data-tfrecord/bert-test/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord', './data-tfrecord/bert-test/part-algo-4-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord']


In [12]:
config = DistilBertConfig.from_pretrained('distilbert-base-uncased',
                                          num_labels=len(CLASSES))
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', 
                                                              config=config)

# Setup the fine-tuning here

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.layers[0].trainable = True
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  3845      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,957,317
Trainable params: 66,957,317
Non-trainable params: 0
_________________________________________________________________


In [14]:
callbacks = []

log_dir = './tensorboard/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks.append(tensorboard_callback)

In [15]:
history = model.fit(train_dataset,
                    shuffle=True,
                    epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=validation_dataset,
                    validation_steps=VALIDATION_STEPS,
                    callbacks=callbacks)

Train for 250 steps, validate for 50 steps
250/250 [==============================] - 416s 2s/step - loss: 1.1174 - accuracy: 0.5820 - val_loss: 0.8488 - val_accuracy: 0.6925


In [16]:
print('Trained model {}'.format(model))

Trained model <transformers.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7f5cf56f7d68>


# Evaluate on Holdout Test Dataset

In [17]:
test_history = model.evaluate(test_dataset,
                              steps=TEST_STEPS,
                              callbacks=callbacks)
print(test_history)

50/50 [==============================] - 21s 410ms/step - loss: 0.8606 - accuracy: 0.6775
[0.8606372255086899, 0.6775]


# Save the Model

In [18]:
model_dir = './fine-tuned'

In [19]:
!mkdir -p $model_dir
model.save_pretrained(model_dir)

In [20]:
!ls -al $model_dir

total 261692
drwxrwxr-x 2 ec2-user ec2-user      4096 Apr 25 18:33 .
drwxrwxr-x 9 ec2-user ec2-user      4096 Apr 25 18:33 ..
-rw-rw-r-- 1 ec2-user ec2-user      1358 Apr 25 18:33 config.json
-rw-rw-r-- 1 ec2-user ec2-user 267959068 Apr 25 18:33 tf_model.h5


In [21]:
cat $model_dir/config.json

{
  "_num_labels": 5,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true

In [22]:
import json
from transformers import TFDistilBertForSequenceClassification

loaded_model = TFDistilBertForSequenceClassification.from_pretrained(model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     })

In [23]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

if NUM_GPUS >= 1:
    inference_device = 0 # GPU 0
else:
    inference_device = -1 # CPU
print('inference_device {}'.format(inference_device))


inference_device -1


In [24]:
from transformers import TextClassificationPipeline
inference_pipeline = TextClassificationPipeline(model=loaded_model, 
                                                tokenizer=tokenizer,
                                                framework='tf',
                                                device=inference_device) # -1 is CPU, >= 0 is GPU

print("""I loved it!  I will recommend this to everyone.""", inference_pipeline("""I loved it!  I will recommend this to everyone."""))
print("""Really bad.  I hope they don't make this anymore.""", inference_pipeline("""Really bad.  I hope they don't make this anymore."""))

I loved it!  I will recommend this to everyone. [{'label': 5, 'score': 0.786064}]
Really bad.  I hope they don't make this anymore. [{'label': 1, 'score': 0.7445503}]


# Uncomment to Start Tensorboard
Let's start Tensorboard and point to the tensorboard logs that we downloaded from S3 directly.

Note:  If you pointed Tensorboard to S3 directly, you must prepend this command with `S3_REGION=[your-region]`.

In [25]:
#%%bash
#pip uninstall -y tensorboard-plugin-wit
#tensorboard --port 6006 --logdir ./tensorboard/ # <== MAKE SURE YOU INCLUDE THE TRAILING `/`

While Tensorboard is running locally on your SageMaker Notebook instance, it is reading the training logs from Amazon S3.

**MAKE SURE YOU CHANGE `[your-region]` BELOW TO YOUR REGION.**

Navigate to https://workshop.notebook.[your-region].sagemaker.aws/proxy/6006/  <== MAKE SURE YOU INCLUDE THE TRAILING SLASH

**MAKE SURE YOU CHANGE `[your-region]` ABOVE TO YOUR REGION.**

_Note:  Make sure you copy the trailing `/` in the link above.  If you see no data, you are likely not using the correct S3 bucket above._

![Tensorboard](img/tensorboard.png)

# Stop Tensorboard
Once you are done, hit `Kernel => Stop` to stop the running `Tensorboard` process in this notebook.